In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.spatial import distance
from scipy.cluster import hierarchy
import os
from IPython.display import display, Markdown
from typing import *
%matplotlib inline  

## Lab 7: Image Analysis

In this lab, you will:
* Load images from files, representing them as arrays.
* Shift images between different color representations.
* Highlight intensity ranges via thresholding.
* Extract a Region of Interest (ROI).
* Create a histogram of image pixel values.
* Use a color histogram to equalize image intensity levels.
* Using a histogram of an ROI, filter the image to show regions sharing the ROI's color.
* Find image features showing the relationship of an ROI to the original image.
* Employ histograms for image clustering.
* Employ feature extraction for image clustering.
* Explore the advantages and disadvantages of these two approaches to image clustering.

We will be using the OpenCV Computer Vision library, imported as `cv2`. OpenCV uses `numpy` (`np`) arrays to represent images in memory. We will use `matplotlib` (which we previously used to show graphs) to display these `numpy` arrays as images in our notebook.

## Step 7.1: Setup

First, we will load our images into memory from files. You should have access to the `CSCI 270 Images 2022` data set. If you have trouble accessing it, you can download the image files from the **Images** directory in the CSCI 270 Team, then upload them to Kaggle to create your own image data set.

We use the `cv2.imread()` function to read each image file into memory as a `numpy` array. The `images` list consists of tuples of each filename along with its `numpy` representation. We use the `show_markdown_table()` function to list all of the filenames, their list indices, and their images sizes.

In [2]:
def show_markdown_table(headers: List[str], data: List) -> str:
    s = f"| {' | '.join(headers)} |\n| {' | '.join([(max(1, len(header) - 1)) * '-' + ':' for header in headers])} |\n"
    for row in data:
        s += f"| {' | '.join([str(item) for item in row])} |\n"
    display(Markdown(s))

In [3]:
prefix = '../input/csci-270-images-2022'
images = [(file, cv2.cvtColor(cv2.imread(f'{prefix}/{file}'), cv2.COLOR_BGR2RGB)) for file in os.listdir(prefix)]
print(f"We have {len(images)} images.")

In [4]:
show_markdown_table(['Index', 'Filename', 'Width', 'Height'], [[i, file, img.shape[1], img.shape[0]] for (i, (file, img)) in enumerate(images)])

Find your image in the list. Then modify the definition of `my_image` in the code block below so that the `plt.imshow()` function displays your image. Currently, it shows the image listed above in row 0.

In [5]:
my_image = images[7][1]
plt.imshow(my_image)

If you would like to adjust the size of your image, use the `figure` function to specify a larger size. Here is an example to make the above image larger. Adjust `figsize` to achieve different sizes.

In [6]:
plt.figure(num=None,figsize=(10,10),dpi=80)
plt.imshow(my_image)

In [7]:
# how to set 'region of interest'
roi = images[7][1][200:450, 800:1100]
plt.imshow(roi)

## Step 7.2: Color Representations and Conversions

There are many different representations of pixel colors. When you load your image, it is in BGR representation. Each pixel is represented as three values: a blue level, a green level, and a red level. The code block below shifts the blue to green, the green to red, and the red to blue. Run the block and see how your image changes.

It may take a while - loops like this in Python are really slow. We won't directly iterate over images very much - OpenCV operations are designed to be very efficient.

In [8]:
def color_shift_left(image):
    shifted = image.copy()
    for row in range(len(shifted)):
        for col in range(len(shifted[row])):
            blue = shifted[row][col][0]
            shifted[row][col][0] = shifted[row][col][1]
            shifted[row][col][1] = shifted[row][col][2]
            shifted[row][col][2] = blue
    return shifted

%time plt.imshow(color_shift_left(my_image))

Write a function called `color_shift_right()` that shifts green to blue, blue to red, and red to green. It will be very similar to `color_shift_left()`, except that the color rotation will be in the other direction.

In [9]:
def color_shift_right(image):
    shifted = image.copy()
    for row in range(len(shifted)):
        for col in range(len(shifted[row])):
            red = shifted[row][col][2]
            shifted[row][col][2] = shifted[row][col][1]
            shifted[row][col][1] = shifted[row][col][0]
            shifted[row][col][0] = red
    return shifted

%time plt.imshow(color_shift_right(my_image))

To convert between color models, we use the `cv2.cvtColor()` function. It takes two arguments:
* The image to be converted.
* The conversion code, indicating the source and destination encoding.

Here are some useful conversion codes:

| Conversion       | Code                   | 
| ---------------- | ---------------------- |
| BGR to grayscale | `cv2.COLOR_BGR2GRAY`   |
| BGR to HSV       | `cv2.COLOR_BGR2HSV`    |
| HSV to BGR       | `cv2.COLOR_HSV2BGR`    |
| BGR to YCrCb     | `cv2.COLOR_BGR2YCrCb`  |
| YCrCb to BGR     | `cv2.COLOR_YCR_CB2BGR` |

We will investigate HSV and YCrCb a little later. For now, we will focus on grayscale. The code block below converts to grayscale.

In [10]:
gray = cv2.cvtColor(my_image, cv2.COLOR_BGR2GRAY)
plt.imshow(gray, cmap='gray', vmin=0, vmax=255)

**Thresholding** an image turns all pixels in the threshold area white, leaving all other pixels black. Run the next two code blocks to see your image thresholded at two different levels.

In [11]:
ret, thresh = cv2.threshold(gray, 50, 255, 0)
plt.imshow(thresh, cmap='gray', vmin=0, vmax=255)
ret

In [12]:
ret, thresh = cv2.threshold(gray, 100, 255, 0)
plt.imshow(thresh, cmap='gray', vmin=0, vmax=255)

## Step 7.3: Region of Interest and ORB Features

To obtain a Region of Interest (ROI), you can slice the `numpy` array storing the image.

In [13]:
peter = my_image[200:800, 800:1100]
plt.imshow(peter)

Examine your image. Select a region that is especially importand or distinctive. Then, in the code block below, create a slice that corresponds to your region of interest, and display it. Try different combinations of slice values until you have captured your ROI to your satisfaction.

In [14]:
my_roi = my_image[700:1150, 400:1000]
plt.imshow(my_roi)

`ORB` is a feature-matching algorithm. It finds distinctive points ("**keypoints**") in an image, forms a model of the area around each of the points, and then finds the best matching corresponding points in another image. We will see if`ORB` can figure out where in your original image your ROI came from.

We will begin by using `ORB` to find the keypoints in the original image and display them. Adjust `figsize` as needed to make the image large enough to see the keypoints, which are highlighted in green.

The variable `kp` contains the coordinates of the keypoints. The variable `des` contains **descriptors**; that is, descriptions of the neighborhood around each keypoint.

In [15]:
orb = cv2.ORB_create()
kp, des = orb.detectAndCompute(my_image, None)
keypoint_image = cv2.drawKeypoints(gray, kp, None, color=(255,0,0), flags=0)
plt.figure(num=None,figsize=(10,10),dpi=80)
plt.imshow(keypoint_image)

Next, adapt the previous code block to find the keypoints in the ROI image `my_roi`. Store the keypoints in a variable named `kp_roi` and the descriptors in a variable named `des_roi`. Converting the ROI image to grayscale prior to viewing the keypoints will make them easier to see. Store the gray-converted roi image in a variable named `gray_roi`.

In [16]:
# YOUR CODE HERE
orb = cv2.ORB_create()
kp_roi, des_roi = orb.detectAndCompute(my_roi, None)
gray_roi = cv2.drawKeypoints(gray, kp, None, color=(255,0,0), flags=0)
plt.figure(num=None,figsize=(10,10),dpi=80)
plt.imshow(keypoint_image)

Now we will find the relationships between the keypoints in the two images. We will use a brute-force matcher that finds the best match between each descriptor in the first and second image. We will then draw the matches between the images. 

Matches are in terms of minimum distance, so sorting the matches will give us the closest matching descriptors first.

The match image produced should show lines connecting the 20 closest matching features. Hopefully, you will see matches between the ROI image and the part of the image from which it originally came.

In [17]:
brute_force = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = sorted(brute_force.match(des, des_roi), key=lambda m: m.distance)
match_image = cv2.drawMatches(my_image, kp, my_roi, kp_roi, matches[:20], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.figure(num=None,figsize=(20,20),dpi=80)
plt.imshow(match_image)

### Keypoint Questions

1. How well did ORB match your ROI back to the original image?

*answer*

2. What aspects of your ROI do you think were helpful to ORB in performing the match?

*answer*

## Step 7.4: Histograms

An image histogram records counts of the numbers of pixels of the given value. This means that for color images, there are separate histograms for each color channel. The code block below creates a histogram for each color channel, and displays all three histograms on a graph. Run the next two code blocks to see your image's color histograms. Then answer the questions in the box that follows.

In [18]:
def plot_histogram_of(image):
    # Adapted from https://docs.opencv.org/4.x/d1/db7/tutorial_py_histogram_begins.html
    color = ('r', 'g', 'b')
    for i, c in enumerate(color):
        hist = cv2.calcHist([image], [i], None, [256], [0, 256])
        plt.plot(hist, color=c)
        plt.xlim([0, 256])
    plt.show()

In [19]:
plot_histogram_of(my_image)

### Histogram Questions 1

1. Which color is the most frequent in your image: Blue, Green, or Red?

*answer*

2. Which color is the least frequent?

*answer*

3. Are these frequencies consistent with how you intuitively understand the colors in the image? Explain your answer, in terms of what you see as the predominant colors (not just green, blue, and red). 

*answer*

Next, we will examine a histogram that splits out intensity (Y) from two color channels (Cr, Cb). The blue curve shows intensity while the red and green curves show the color channels.

In [20]:
plot_histogram_of(cv2.cvtColor(my_image, cv2.COLOR_BGR2YCrCb))

### Histogram Questions 2

How well does the intensity level correspond with your intuitive idea about the relative brightness/darkness of your image?

*answer*

We can use the intensity band from the `YCrCb` histogram to equalize the intensity levels of an image. To achieve this, we:
* Convert the BGR image to YCrCb.
* Get the Y values.
* Use OpenCV's `equalizeHist()` function to equalize the Y levels.
* Merge the equalized levels back with the CrCb values.
* Convert the merged YCrCb image back to BGR and view.

Run the code block below, then answer the questions that follow.

In [21]:
# Histogram equalization in color
# Transition to YCrCb domain, then equalize on Y
# Adapted from: https://stackoverflow.com/questions/42651595/histogram-equalization-python-for-colored-image

y_cr_cb = cv2.cvtColor(my_image, cv2.COLOR_BGR2YCrCb)
y, cr, cb = cv2.split(y_cr_cb)
y_eq = cv2.equalizeHist(y)
y_cr_cb_eq = cv2.merge((y_eq, cr, cb))
plt.imshow(cv2.cvtColor(y_cr_cb_eq, cv2.COLOR_YCR_CB2BGR))


### Histogram Questions 3

1. How drastic a change to your image does histogram equalization represent?

*answer*

2. Does the change increase or decrease the aesthetic appeal of your image?

*answer*

3. In general, under what circumstances do you consider this technique to be of greatest value?

*answer*

We can also use a histogram to create a color filter we can use for thresholding. Select a region of your image that has a distinctive coloration pattern in the code block below.

In [22]:
# FILL IN SLICE COORDINATES BELOW
color_roi = my_image[200:800, 800:1100]
plt.imshow(color_roi)

We will use this ROI to create a **histogram backprojection** of the image. The backprojection has higher intensity levels to regions where the color is a good match to the histogram, and lower intensity levels to regions where the color is not as good a match.

This technique works best using an HSV (Hue, Saturation, Brightness) image, ignoring the Brightness component.

Run the code block below and answer the questions that follow.

In [23]:
# https://docs.opencv.org/4.x/dc/df6/tutorial_py_histogram_backprojection.html
img_hsv = cv2.cvtColor(my_image, cv2.COLOR_BGR2HSV)
subimg_hsv = cv2.cvtColor(color_roi, cv2.COLOR_BGR2HSV)

subimg_hist = cv2.calcHist([subimg_hsv], [0, 1], None, [180, 256], [0, 180, 0, 256])
cv2.normalize(subimg_hist, subimg_hist, 0, 255, cv2.NORM_MINMAX)
back = cv2.calcBackProject([img_hsv], [0, 1], subimg_hist, [0, 180, 0, 256], 1)

disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
cv2.filter2D(back, -1, disc, back)

ret, thresh = cv2.threshold(back, 50, 255, 0)
thresh = cv2.merge((thresh, thresh, thresh))
filtered = cv2.bitwise_and(my_image, thresh)

both = np.vstack((thresh, filtered))
plt.figure(figsize=(10,10))
plt.imshow(both)

### Histogram Questions 4

1. Look at the juxtaposed threshold image and filtered image. How well is your ROI's color represented in the final image?

*answer*

2. What are some practical applications you can envision of this technique?

*answer*

## Step 7.5 Histogram Clustering

We can use color histograms to cluster the images in our data set. The next code block defines functions to display a dendrogram of the clustered images, using the Cosine distance to determine the clusters. The following code block creates histograms of each image in our data set. 

Run both code blocks and answer the questions that follow. Bear in mind that it may take a minute or two to run the clustering algorithm.

In [24]:
def dendrogram_from(names, distances, title):
    condensed = distance.squareform(distances)
    c = hierarchy.linkage(condensed, 'average')
    plt.figure()
    plt.figure(figsize=(10, 10))
    dn = hierarchy.dendrogram(c, labels = names, leaf_rotation=90)
    plt.title(f'UPGMA Clustering: {title}')
    plt.ylabel("Total Distance")
    plt.show()
    
def cluster_1d(names, images1d, title, distance_func):
    distances = [[distance_func(a, b) for a in images1d] for b in images1d]
    dendrogram_from(names, distances, title)

In [ ]:
names = [i[0] for i in images]
histograms = [cv2.calcHist([i[1]], [0, 1, 2], None, [256] * 3, [0, 256] * 3).reshape(-1, 1) for i in images]
%time cluster_1d(names, histograms, "Histograms Cosine", distance.cosine)

### Clustering Questions 1

1. Select a few images that are marked as similar, and write some code blocks to view them. Which images did you choose? Intuitively speaking, do you find them to be similar?

*answer*

2. Select a few images that are marked as not similar, and write some code blocks to view them. Which images did you choose? Intuitively speaking, do you find them to be dissimilar?

*answer*

3. Overall, how well does the Cosine distance of histograms serve as a clustering technique? In other words, to what degree do the clusters match your intuition about the similarity of the images?

*answer*

## Step 7.6 Feature Clustering

We can use `ORB` feature detection as a clustering technique as follows:
* Find all the ORB keypoints and descriptors for all of our images.
* Create a distance matrix as follows:
  * For each pair of images:
    * Find their keypoint matches, and sort them.
    * Add up the top 20 distances of the closest matches. This will be the distance between the two images.
* Call `dendrogram_from()` using that distance matrix.

Write the function `get_orb_matches()` to find the sorted ORB matches between two images. Then write `orb_distances()` to create the ORB distance matrix. (It should call `get_orb_matches()` as a subroutine.) Then run the following code block to view the resulting clusters.

In [ ]:
def get_orb_matches(kp1, des1, kp2, des2):
    #find all the sorted orb matches between two images
    x = 1
    
def orb_distances(images, num_matches):
    kp_list = []
    des_list = []
    for img in images:
        orb = cv2.ORB_create()
        kp, des = orb.detectAndCompute(img, None)
        kp_list.append(kp)
        des_list.append(des)

# i am so sorry, but i am going to use another token to do revisions on this lab

In [ ]:
dendrogram_from(names, orb_distances(images, 20), "ORB Distances (20 matches)")

### Clustering Questions 2

1. Select a few images that are marked as similar, and write some code blocks using `get_orb_matches()` to view their keypoint correspondences. Which images did you choose? Intuitively speaking, do you find them to be similar?

*answer*

2. Select a few images that are marked as not similar, and write some code blocks to view them as in question 1. Which images did you choose? Intuitively speaking, do you find them to be dissimilar?

*answer*

3. Overall, how well does the ORB feature distance serve as a clustering technique? In other words, to what degree do the clusters match your intuition about the similarity of the images?

*answer*

4. Compare and contrast histogram distance and ORB feature distance as mechanisms for clustering. What advantages and disadvantages does each scheme have?

*answer*